In [1]:
import os
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns

from dataset.OnlineDataset import EnergyDataset
import torch

In [8]:
ds = EnergyDataset(raw_data_path='../data/1_processed',
                   sliding_window_size=336,
                   sliding_window_offset=336,
                   forecast_size=0,
                   customer=1,
                   mode='test',
                   device='cpu')

# prosumption = torch.tensor([])
# price = torch.tensor([])
# for i in range(0,1):
#     prosumption = torch.cat([prosumption, ds[i]['prosumption']])
#     price = torch.cat([price, ds[i]['price']])
# T = len(prosumption)

In [12]:
ds[0]

TensorDict(
    fields={
        load: Tensor(shape=torch.Size([336]), device=cpu, dtype=torch.float32, is_shared=False),
        price: Tensor(shape=torch.Size([336]), device=cpu, dtype=torch.float32, is_shared=False),
        prosumption: Tensor(shape=torch.Size([336]), device=cpu, dtype=torch.float32, is_shared=False),
        pv: Tensor(shape=torch.Size([336]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([336]),
    device=cpu,
    is_shared=False)

# WO Battery

In [14]:
cost = 0

for i in range(1):
    curr_ds = ds[i]
    prosumption = curr_ds['prosumption']
    price = curr_ds['price']
    for j in range(336):

        cost  += prosumption[j] * price[j] if prosumption[j]>=0 else prosumption[j]*0.1
cost

tensor(54.3945)

# Gurobi Optimizer

In [16]:
curr_ds = ds[0]
prosumption = curr_ds['prosumption']
price = curr_ds['price']
T=336
max_charge = 1.25
max_discharge = -1.25
battery_capacity = 10.0
big_M = 1e4  # A sufficiently large number

# Create model
model = gp.Model("BatteryCharging")
model.Params.OutputFlag = 1

# Variables
action = model.addVars(T, lb=max_discharge, ub=max_charge, name="action")
soe = model.addVars(T + 1, lb=0.0, ub=battery_capacity, name="soe")
z = model.addVars(T, lb=0.0, ub=10, vtype=GRB.CONTINUOUS, name='actual_price')
b = model.addVars(T, vtype=GRB.BINARY, name="b")

# Initial SoE
model.addConstr(soe[0] == 0.0, name="initial_soe")

eps = 0.0001
M = 20 + eps

for t in range(T):
    # Battery dynamics
    model.addConstr(soe[t + 1] == soe[t] + action[t], name=f"soe_update_{t}")
    model.addConstr(action[t]+prosumption[t] >= 0 + eps - M * (1 - b[t]), name="bigM_constr1")
    model.addConstr(action[t]+prosumption[t] <= 0 + M * b[t], name="bigM_constr2")

    # Add indicator constraints
    model.addConstr((b[t] == 1) >> (z[t] == price[t]), name="indicator_constr1")
    model.addConstr((b[t] == 0) >> (z[t] == 0.1), name="indicator_constr2")



# Objective function: minimize cost
obj = gp.quicksum(
    (prosumption[t] + action[t]) * z[t]
    for t in range(T)
)
model.setObjective(obj, GRB.MINIMIZE)

# Optimize
model.optimize()

if model.status == GRB.OPTIMAL:
    final_cost = 0.0
    actions = [action[t].X for t in range(T)]
    soes = [soe[t].X for t in range(T+1)]
    used_price = [z[t].X for t in range(T)]
    for t in range(T):
        curr_price = price[t] if (prosumption[t]+actions[t]) >= 0 else 0.1
        final_cost += (prosumption[t]+actions[t])*curr_price

    print(f'final cost: {final_cost}')

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 3700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1009 rows, 1345 columns and 2353 nonzeros
Model fingerprint: 0xacf52e4d
Model has 336 quadratic objective terms
Model has 672 simple general constraints
  672 INDICATOR
Variable types: 1009 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-02, 3e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e-02, 2e+01]
  GenCon rhs range [1e-01, 5e-01]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 101 rows and 389 columns
Presolve time: 0.03s
Presolved: 1195 rows, 1243 columns, 3011 nonzeros
Variable types: 956 continuous, 287 integer (287 binary)
Found heuristic solution: objective 41.171